#<div style="display:block">
    <div style="width: 20%; display: inline-block; text-align: left;">
        <img src="http://upload.wikimedia.org/wikipedia/en/0/0c/Mu_Sigma_Logo.jpg" style="height:75px; margin-left:0px" />
    </div>
    <div style="width: 59%; display: inline-block">
        <h1  style="text-align: center">Keyterms Extraction: TF-IDF + POS Tags + InD Wiki + Word2Vec Similarity Experiments </h1>
        <div style="width: 100%; text-align: center; display: inline-block;"><i>Author:</i> <strong>Pavan Kumar</strong> </div>
    </div>
    <div style="width: 20%; text-align: right; display: inline-block;">
        <div style="width: 100%; text-align: left; display: inline-block;">
            <i>Created: </i>
            <time datetime="2014-03-10" pubdate>19th Sep 2016</time>
            <i>Modified: </i>
            <time datetime="2014-03-10" pubdate>26th Sep 2016</time>
        </div>
    </div>
</div>

## Hypothesis
Based on  <a href="http://qa.ird.mu-sigma.com/user/iot/notebooks/M.R.%20Robot/Ears/Keyterms%20Extraction%20-%20Community%20Cluster.ipynb"> previous works </a>, we concluded that community clustering was not producing promising results, as the clusters detected were not logically related. The <a href="http://qa.ird.mu-sigma.com/user/iot/notebooks/M.R.%20Robot/Keyword%20Extraction%20-%20Literature%20Survey.ipynb"> literature survey led us to the next possible approach - to use a <a href = "http://scikit-learn.org/stable/modules/feature_extraction.html#tfidf-term-weighting"> scikit-learn implementation</a> of calculating TF-IDF measure to further filter out n-grams extracted from the text. The n-grams were words in the text that appeared as article titles on wikipedia. As an addition we have also added InD wikipedia data to this method to incorporate data about in-house artifacts (muESP, muStream etc.). Finally after the semantic relatedness graph is generated (based on links that appear between two wikipedia titles), we add words in a sentence relate to or explain the candidate n-grams further to the graph generated. For example, if the candidate term is either a noun or a verb (specified by its <a href = "https://en.wikipedia.org/wiki/Part-of-speech_tagging">POS tag</a>), we choose to add words that describe the candidate term (adjective, adverbs and quantifiers) to the candidate graph. The <a href = "https://research.googleblog.com/2016/05/announcing-syntaxnet-worlds-most.html" >Parsey McParseface</a> model developed by Google on the <a href = "https://github.com/tensorflow/models/tree/master/syntaxnet">SyntaxNet framework</a> provides the solution we are ooking for in terms of extracting POS (Part-of-speech) tags and the dependency tree for the sentence. Rest of the core functionality of Keyword Extraction remains the same.

### Step 1: Loading Wikipedia database
Our first approach of crawling wikipedia to extract titles, links and categories proved to be inefficient and very slow. We then downloaded the entire wikipedia dump from <a href="https://en.wikipedia.org/wiki/Wikipedia:Database_download#English-language_Wikipedia">Wikipedia:Database download.</a> We then used <a href = "http://www.cs.technion.ac.il/~gabr/resources/code/wikiprep/">WikiPrep</a> to preprocess the data and obtain a more descriptive XML representation of the database. We also incorporated InD wiki data by scraping the InD wiki for titles, links and categories as described <a href ="http://ird.mu-sigma.com/wiki/index.php/Scrape_I%26D_wiki_as_well_for_MR_Robot_report_generation"> here </a>.



In [30]:
from collections import Counter, OrderedDict
from nltk.corpus import stopwords
import xml.etree.cElementTree as ET
from operator import itemgetter
import cPickle as pickle
from sklearn.feature_extraction.text import TfidfVectorizer
import itertools
import subprocess
import numpy as np
import pandas as pd
import operator
import xml.sax
import igraph
import string
import numpy
import time
import nltk
import json
import math
import json
import sys
import re
import os
import gensim
import re

if not os.path.exists("savepoints/"):
    os.makedirs("savepoints/")
### TODO: Move all this to config file
## File configs 
#--------------------------------------------------------------------------------------------------#
INPUT_TRANSCRIPT = "data/sample_text.txt"
TITLE_DATA_LOCATION = "data/enwiki-latest-all-titles-in-ns0"
EXTENDED_XML_LOCATION = "data/20051105_pages_articles.hgw.xml"
CATEGORY_TREE_LOCATION = "data/20051105_pages_articles.cat_hier"
WIKI_TITLES_PICKLE = "data/wiki_titles.p"
IND_WIKI_LOCATION = "data/ind_wiki.p"
CATEGORY_TREE_PICKLE = "data/category_tree.p"
CATEGORY_TREE_STARTLINE = 3
#---------------------------------------------------------------------------------------------------#

## Algo configs
#Link Heuristic
#--------------------------------------------------
SEE_ALSO_SCORE = 5.0
INV_SEE_ALSO_SCORE = 4.0
DOUBLE_LINK_SCORE = 4.0
COMMON_CATEGORY_SCORE = 2.0
REGULAR_LINK_SCORE = 3.0
INV_REGULAR_LINK_SCORE = 2.0
POS_SCORE_0 = 1.0
#---------------------------------------------------
TOP_N_PERCENT = 10
SENTENCE_WINDOW = 10 #https://insidegovuk.blog.gov.uk/2014/08/04/sentence-length-why-25-words-is-our-limit/
SYNTAXNET_LOCATION = "/home/iot/models/syntaxnet/"
COMCLUSTER_LOCATION = os.getcwd()
SYNTAXNET_OUTPUT_FILE = "savepoints/syntaxnet_output"
WORD2VEC_MODEL_FILE = "data/GoogleNews-vectors-negative300.bin"
WORD2VEC_STORED_MODEL = "data/word2vec_model"
CORENLP_LOCATION = "data/stanford-corenlp-full-2015-12-09/"
SENTENCE_DOC = "savepoints/sentence_doc"

#Dict to store wiki titles (in memory)
wiki_titles = {}

#Dict to store ind wiki data
ind_wiki = {}

#Dict to store entire category tree (in memory)
category_tree = {}

#Final set of edges
final_set_edges = []

#Variables to find page_id, title, links and categories of pages
title = None
page_id = None
links = None
categories = None
new_element = None


#Miscellaneous Variables
line_no = 1  #current line number being processed any file.



In [31]:
####################################################################################################
# Function 
#     Name:   wikidata
#
# Task:       This function is used to load the information from wikipedia dumps. This is fairly slow
#             and should run with 'new_wikidata' flag as TRUE only if the wiki files are new.
#
# Parameters:-
#             1. title_location:           Location (absolute path) of the file 
#                                          (Ex: 'enwiki-latest-all-titles-in-ns0') 
#                                          which stores all wiki titles.
#
#             2. category_tree_location:   Location (absolute path) of the file 
#                                          (Ex: 20051105_pages_articles.cat_hier)
#                                          which stores wikipedia's category tree.
#
#             3. category_tree_start_line: Line number in the category tree file when the data starts.
#                                          First line is the column titles and next line is empty. 
#             
#             4. new_wikidata:            If the wiki data was updated since last run.
def wikidata(title_location = TITLE_DATA_LOCATION, 
             category_tree_location = CATEGORY_TREE_LOCATION,
             category_tree_start_line = CATEGORY_TREE_STARTLINE,
             use_savepoint = True):

    global wiki_titles
    global category_tree
    global line_no
    global ind_wiki

    print "Loading Wikipedia Data..."


    if not(use_savepoint):
        #------------_Read once and store in pickle ---------------#
        with open(title_location, "r") as title_file:
            for line in title_file:
                wiki_titles[line.decode('ascii','ignore').strip("\n").lower()] = ''
        
        with open(category_tree_location, "r") as category_file:
            for line in category_file:
                if line_no > category_tree_start_line:
                    line = line.split()
                    category_tree[line[0]] = line[1:]
                line_no += 1

        #Savepoint
        pickle.dump(wiki_titles, open("savepoints/wiki_titles.p","wb"))
        pickle.dump(category_tree, open("savepoints/category_tree.p","wb"))

    else:
        wiki_titles = pickle.load(open(WIKI_TITLES_PICKLE, "rb"))
        category_tree = pickle.load(open(CATEGORY_TREE_PICKLE,"rb"))

    #IND wiki data
    ind_wiki = pickle.load(open(IND_WIKI_LOCATION))

    print "Size of wiki titles in memory: " + str(float(sys.getsizeof(wiki_titles))/(1024 * 1024)) + " MB"
    print "Sample Title: " +str(wiki_titles.keys()[150001]) +"\n"

    return wiki_titles, category_tree

### Step 2: Extracting article titles from meeting transcripts
We extract all individual words and n-grams (upto 5-grams) which match a wikipedia article title from the meeting transcripts and label them as 'candidate keyterms'.

In [32]:
####################################################################################################
# Function 
#     Name:   extract_ngrams
#
# Task:       This function is used to extract ngrams and individual words (individual words and n-grams) 
#             from input transcript file.

# Parameters:-
#             1. transcript_file: Absolute path of the meeting transcript file being processed.
def extract_ngrams(transcript_file = INPUT_TRANSCRIPT):


    # List of all n-grams
    words = []
    bigrams = [] 
    trigrams = [] 
    fourgrams = []
    fivegrams = []
    
    # Reading Transcript
    with open(transcript_file) as book:
        sample_text = book.read()

    # Remove punctuation, numbers and replace duplicate spaces
    replace_punctuation = string.maketrans(string.punctuation, " "*len(string.punctuation)) 
    sample_text = sample_text.translate(replace_punctuation)
    sample_text = re.sub("[^a-zA-Z]", " ", sample_text)
    sample_text = re.sub(" +", " ", sample_text)

    # Extract words from raw data
    tokenized_sample_text = tokenize(sample_text)
    word_count = len(tokenized_sample_text)
    filtered_text = [word.lower() for word in tokenized_sample_text if word.lower() not in stopwords.words('english')]

    for w in filtered_text:
        if(w in wiki_titles or w in ind_wiki.keys()):
            words.append(w)

    #n-grams are filtered out if they are not article titles or don't redirect to an article. 
    for i in xrange(word_count):
        page_found = True # Check if a queried page exists.

        if(i>0):

            phrase = tokenized_sample_text[i-1] +"_"+ tokenized_sample_text[i]
            if(phrase in wiki_titles or phrase in ind_wiki.keys()):
                bigrams.append(phrase)
                # print "Bigram Article Found: "+str(phrase)

            # print phrase

        if(i>1):

            phrase = tokenized_sample_text[i-2] +"_"+ tokenized_sample_text[i-1] +"_"+ tokenized_sample_text[i]
            # print phrase
            if(phrase in wiki_titles or phrase in ind_wiki.keys()):
                trigrams.append(phrase)
                # print "Trigram Article Found: "+str(phrase)


        if(i>2):

            phrase = tokenized_sample_text[i-3] +"_"+ tokenized_sample_text[i-2] +"_"+ tokenized_sample_text[i-1] +"_"+ tokenized_sample_text[i]
            # print phrase
            if(phrase in wiki_titles or phrase in ind_wiki.keys()):
                fourgrams.append(phrase)
                # print "Fourgram Article Found: "+str(phrase)



        if(i>3):

            phrase = tokenized_sample_text[i-4] +"_"+ tokenized_sample_text[i-3] +"_"+ tokenized_sample_text[i-2] +"_"+ tokenized_sample_text[i-1] +"_"+  tokenized_sample_text[i]
            # print phrase
            if(phrase in wiki_titles or phrase in ind_wiki.keys()):
                fivegrams.append(phrase)
                # print "Fivegram Article Found: "+str(phrase)

    # Dump to pickle load it - Savepoint
    candidate_terms = words + bigrams + trigrams + fourgrams + fivegrams

    freq_dist = Counter(candidate_terms)
    final_candidate_terms = []

    for term in freq_dist:
        if('_' in term):
            final_candidate_terms.append(term)

        elif(freq_dist[term]>1):
            final_candidate_terms.append(term)

    candidate_terms = final_candidate_terms

    #SAVEPOINT
    print str(len(candidate_terms))+" n-grams extracted:\n"+ str(candidate_terms)
    pickle.dump(candidate_terms, open("savepoints/candidate_terms.p", "wb"))
    return candidate_terms


### Step 3: TF-IDF Filtering
We choose to keep only the top n% (n can be optimized after experimentation) candidate n-grams that are most relevant to the document (TF) and topic of discussion (IDF).

In [33]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    # stems = stem_tokens(tokens, stemmer)
    # return stems
    return tokens 



####################################################################################################
# Function 
#     Name:   top_n_tfidf
#
# Task:       Filter out candidate keywords based on TF-IDF value. The top n% of the candidate
#             terms are kept. 

# Parameters:-
#             1. candidate_terms: n_grams extracted.
#             2. sample_transcripts_path: Folder containing all the transcripts (for calc IDF). 
#             3. transcripts_file: Transcript file path.

def top_n_tfidf(candidate_terms, sample_transcripts_path ="data/sample_transcripts", transcript_file = INPUT_TRANSCRIPT):
    top_n = TOP_N_PERCENT
    token_dict = {}
    tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english') #Data structure to store the TFIDF values as feature vectors

    candidate_terms_expanded = [term.split('_') for term in candidate_terms]
    chain = itertools.chain(*candidate_terms_expanded)
    candidate_terms_expanded = list(chain)
    print "Expanded candidate_terms:"
    print candidate_terms_expanded

    #Reading from sample transcripts - TODO: Do this on a large number of files and store the tf_idf values in pickle
    for subdir, dirs, files in os.walk(sample_transcripts_path):
        for file in files:
            file_path = subdir + os.path.sep + file
            print "Reading "+str(file_path)+" ..."
            trans = open(file_path, 'r')
            text = trans.read()
            lowers = text.lower()
            no_punctuation = lowers.translate(None, string.punctuation) 
            token_dict[file] = no_punctuation

    tfs = tfidf.fit_transform(token_dict.values())

    transcript = str(open(transcript_file,'r').read().lower().translate(None, string.punctuation))

    response = tfidf.fit_transform([transcript]) #Need to pickle this and load back as it grows.
    pickle.dump(tfidf, open("tf_idf.p","wb"))
    feature_names = tfidf.get_feature_names()

    freq_list = []
    temp = set()
    for col in response.nonzero()[1]:
        freq_list.append(tuple( (feature_names[col].encode('ascii','ignore'), response[0, col] )) )

    top_n_words = [x for x in sorted(freq_list, key = itemgetter(1))] #Sorting acc to tfidf value
    top_n_words = [ x for x in top_n_words if x[0] not in stopwords.words('english')] #Removing stop words
    top_n_words = top_n_words[::-1]
    top_n_words = top_n_words[:int(round(len(top_n_words)*top_n/100))]
    print "TOP_N_WORDS (TF-IDF): "+str(len(top_n_words))
    print top_n_words

    for x in top_n_words:
        for y in candidate_terms:
            if(x[0] in y.split("_")):
                temp.add(y) 
    
    candidate_terms = [x for x in temp]
    # candidate_terms = candidate_terms[:int(round(len(candidate_terms)*top_n/100))]
    print "Top "+str(len(candidate_terms))+" words with highest TF-IDF:"
    print candidate_terms
    return candidate_terms


### Step 4: Extract word similarity using word2vec model
Between non-pos words, word2vec similarity is used to later specify the link distance in the graph visualization. 

In [34]:
##############################################################################################
# Function 
#     Name:   word2vec_similarity
#
# Task:        This function calculates similarity between all the words in the final candidate
#              graph using word2vec model. This similarity score is used to specify the distance
#              between nodes.
#              http://mccormickml.com/2016/04/12/googles-pretrained-word2vec-model-in-python/
#
# Parameters:- 1. candidate_graph: Terms between which similarity measure is to be extracted.
#              2. word2vec_model: File which stores google's word2vec model.
def word2vec_similarity(candidate_graph, word2vec_model = WORD2VEC_MODEL_FILE):
    word_similarities = {}
    word_list = [x for x in candidate_graph.vs()["name"]]
    word_list_new = []

    print "Loading word2vec model ..."
    time1 = time.time()
    if(os.path.exists(WORD2VEC_STORED_MODEL)):
        model = gensim.models.Word2Vec.load(WORD2VEC_STORED_MODEL,mmap="r")
    else:
        model = gensim.models.Word2Vec.load_word2vec_format(WORD2VEC_MODEL_FILE, binary=True)
        model.init_sims(replace=True)
        model.save(WORD2VEC_STORED_MODEL)

    print "Size of the word2vec loaded into memory : ", str(float(sys.getsizeof(model))/(1024 * 1024)), "MB"
    
    # print "Word List:\n\t", word_list
    for word in word_list:           
        if(" (" in word):
            word_split = word.split(" ")
            # print "\tSplitting word:", word_split
            if(" (" not in word_split[-1]):
                word_list_new.append(word)

        else:
            word_list_new.append(word)

    # print "Word List Updated: "
    print word_list_new

    for term1 in word_list_new:
        word_similarities[term1] = {}
        words1 = term1.split("_")

        for term2 in word_list_new:
            if(term1 != term2):
                words2 = term2.split("_")
                score = []

                for w1 in words1:
                    for w2 in words2:
                        try:
                            score.append(model.similarity(w1.split(" ")[0], w2.split(" ")[0]))

                        except KeyError as e:
                            score.append(0.0)
                            #print str(e).split(":")[-1], "not found in word2vec model."
                            pass

                score_avg = sum(score)/len(score) # Taking average score
                word_similarities[term1][term2] = score_avg

    # print json.dumps(word_similarities, sort_keys=True, indent = 2)
    time2=time.time()
    print "\n\nWord2Vec Run Time:"+ str(round(time2-time1, 4)) + " seconds." 
    return word_similarities


### Step 5: Prepare the category and link map to generate relatedness graph
For each term in the candidate set, the title is mapped to the category it belongs to, other articles it is linked to and its 'See Also' section. This will be used to calculate semantic relatedness. Using the relatedness score as the weight of the edge, a graph is generated based on wikipedia links that appear between each keyword article title (which are represnted as nodes). This section also involves extracting POS tags to add words that describe a keyword is also added to this graph. 

In [43]:

##############################################################################################
# Function 
#     Name:   build_graph
#
# Task:        Build a graph where nodes represent the candidate terms and the edge weights 
#              represent the relatedness of the connected nodes. Next, find communities within
#              the constructed graph using a greedy implementation of the Girvan-Newman on large
#              networks.
#
# Parameters:- 1. Candidate_terms: Terms from which the graph is to be generated.
#              2. transcript_file: Absolute path of the meeting transcript file being processed.
def build_graph(candidate_terms, extended_xml_location = EXTENDED_XML_LOCATION):

    #Variables to find page_id, title, links and categories of pages
    global title
    global page_id
    global links
    global categories
    global new_element

    #Generating csv for viz
    link_edges = {'source':[], 'target':[], 'relation':[]}
    nodes = {'name':[], 'community':[], 'size':[]}
    colour = 0

    candidate_graph = igraph.Graph()

    #Create link and category map from wikipedia data
    for event, elem in ET.iterparse(EXTENDED_XML_LOCATION):
        
        if (elem.tag == "page"):
            new_element = False
            title = None
            categories = None
            links = None
            page_id = elem.attrib["id"]

        elif(elem.tag == "title"):

            if(elem.text):
                title = elem.text.encode('ascii','ignore').lower().replace(" ", "_")

                if (title in candidate_terms):
                    new_element = True
                    colour += 1
                    

                    candidate_graph.add_vertex(name = title, id = page_id, 
                                                categories = "", links = "", see_also= "",colour = colour)

                    # print "    Candidate term \"" + str(title)+"\" found"

                else:
                    new_element = False

            # print "    title: " + str(title)
            elem.clear()
            continue

        elif(elem.tag == "categories" and new_element):
            if(elem.text): 
                categories = elem.text.encode('ascii','ignore').split()
                candidate_graph.vs.find(name=title)['categories'] = categories
            else: 
                candidate_graph.vs.find(name=title)['categories'] = ''

        elif(elem.tag == "links" and new_element):
            if(elem.text):
                links = elem.text.encode('ascii','ignore').split()
                candidate_graph.vs.find(name=title)['links'] = links
            else: 
                candidate_graph.vs.find(name=title)['links'] = ''

        elif(elem.tag == "h1" and elem.text == "See also" and new_element):
            if(elem.tail):
                see_also = elem.tail.encode('ascii','ignore').replace("*","").split("\n")
                candidate_graph.vs.find(name=title)['see_also'] = [x for x in see_also if len(x)>0]
            else:
                candidate_graph.vs.find(name=title)['see_also'] = links
        
        elem.clear()

    #Use Wiki data
    for ind_title in ind_wiki.keys():
        if ind_title in candidate_terms  and  ind_title not in candidate_graph.vs():
            print "IND WIKI TITLE DETECTED"
            candidate_graph.add_vertex(name=ind_title, id=ind_title,
                                        links=ind_wiki[ind_title]["links"], see_also="",
                                        categories=ind_wiki[ind_title]["categories"], colour=colour)
            colour+=1

    #Calculate relatedness using wiki links
    if len(candidate_graph.vs())> 0: #Checking if any words were detected.

        candidate_terms = [ term for term in candidate_graph.vs()["name"] ]
        max_relation = 0.0
        for node1 in candidate_terms:
            for node2 in candidate_terms:
                relation = 0.0
                if(node1 == node2):
                    continue
                
                node1_page_id = candidate_graph.vs.find(name =node1)['id']
                node1_categories = candidate_graph.vs.find(name =node1)['categories']
                node1_links = candidate_graph.vs.find(name =node1)['links']

                if candidate_graph.vs.find(name =node1)['see_also']!= '':
                    node1_see_also = candidate_graph.vs.find(name =node1)['see_also']
                else: node1_see_also = None

                node2_page_id = candidate_graph.vs.find(name =node2)['id']
                node2_categories = candidate_graph.vs.find(name =node2)['categories']
                node2_links = candidate_graph.vs.find(name =node2)['links']

                if candidate_graph.vs.find(name =node2)['see_also']!= '':
                    node2_see_also = candidate_graph.vs.find(name =node2)['see_also']
                else: node2_see_also = None


                #Check See also
                if node1_see_also and (node2 in node1_see_also):
                    relation += SEE_ALSO_SCORE

                #Check Inverse see also
                if node2_see_also and (node1 in node2_see_also):
                    relation += INV_SEE_ALSO_SCORE

                #Check Double Link
                if (node1_page_id in node2_links) and (node2_page_id in node1_links):
                    relation += DOUBLE_LINK_SCORE

                #Check Regular Link
                if node2_page_id in node1_links:
                    relation += REGULAR_LINK_SCORE

                #Checking Inv Regular link
                if node1_page_id in node2_links:
                    relation += INV_REGULAR_LINK_SCORE

                #Checking for common category
                #TODO Implement WikiRelate! concepts- Can use dampening for increase in path length.
                common_categories =  len([common_category for common_category in node1_categories if common_category in node2_categories])
                if common_categories > 0:
                    relation += common_categories*COMMON_CATEGORY_SCORE

                #Add edge to dict
                if(relation>0.0 and candidate_graph.get_eid(node1, node2, directed=False, error=False) == -1):    
                    candidate_graph.add_edge(source = node1, target = node2, relation=relation)
                    max_relation = max(relation, max_relation)

        #Deleting vertices not in any edge endpoints
        candidate_graph.delete_vertices(candidate_graph.vs.select(_degree=0))

        #Adding POS relations 
        candidate_terms = [term for term in candidate_graph.vs()["name"]]
        word_dependency, sentiments = extract_pos(INPUT_TRANSCRIPT,candidate_terms)
        for key in sorted(word_dependency.keys()):
            for word in word_dependency[key]:
                if(word and (word[0]+" ("+word[1].lower()+")" not in candidate_graph.vs()["name"] and key in candidate_graph.vs()["name"])):
                    # print word[0], "added." 
                    candidate_graph.add_vertex(name = word[0]+" ("+word[1].lower()+")", colour = candidate_graph.vs.find(name=key)['colour'] )
                    candidate_graph.add_edge(source = key, target = word[0]+" ("+word[1].lower()+")", relation=POS_SCORE_0)

        #Deleting vertices not in any edge endpoints
        candidate_graph.delete_vertices(candidate_graph.vs.select(_degree=0))

        #Extract similarity measure for each keyword.
        word_similarity = word2vec_similarity(candidate_graph)


        for vertex in candidate_graph.vs():
            nodes['community'].append(vertex['colour'])
            nodes['name'].append(vertex['name'])
            nodes['size'].append(vertex.degree())

        for edge in candidate_graph.es():
            link_edges['source'].append(edge.source)
            link_edges['target'].append(edge.target)

            if("(" in str(candidate_graph.vs()[edge.source]['name']).split(" ")[-1] or "(" in str(candidate_graph.vs()[edge.target]['name']).split(" ")[-1]):
                link_edges['relation'].append(POS_SCORE_0)

            else:
            #Adding word2vec similarity instead of relation 
                similarity = (word_similarity[candidate_graph.vs()[edge.source]['name']][candidate_graph.vs()[edge.target]['name']])
                similarity = similarity + 1


                relation = (edge["relation"]) * similarity
                

                link_edges['relation'].append(relation)

        # for link_source, link_target, link_weight in itertools.izip(link_edges['source'], link_edges['target'], link_edges['relation']):
        #     print nodes['name'][link_source], "==>", nodes['name'][link_target], "Weight:", nodes['name']['relation']

        ##Savepoint
        print(candidate_graph.summary())
        candidate_graph.write_pickle("savepoints/candidate_graph.p")
        print "Savepoint - Candidate Graph"

        pd.DataFrame.from_dict(nodes).to_csv("savepoints/nodes.csv")
        pd.DataFrame.from_dict(link_edges).to_csv("savepoints/links.csv")  
        
        return candidate_graph

    else:
        pd.DataFrame.from_dict(nodes).to_csv("savepoints/nodes.csv")
        pd.DataFrame.from_dict(link_edges).to_csv("savepoints/links.csv")  
        print "No Wiki titles could be extracted from the n-grams."
        return None



### Step 6: Extract POS tags
We extract POS tags for each sentence that contains a keyword. All the adjectives, adverbs and quantifiers describing the keyowrd in question (if the keyword is a noun or a verb) are added to the relatedness graph. All the nouns and verbs the keyword (which maybe an adjective or adverb) may describe is also added.


In [44]:
####################################################################################################
# Function 
#     Name:   parse_syntaxnet_output
#
# Task:       This method reads the file that the syntaxnet output was written to, and parses 
#             that into memory.
#           
# Parameters:-
#             1. output_text: Absolute path of the syntaxnet output file being processed.
#             2. key_sentence_map: Dict in which sentences where keywords appears. 

def parse_syntaxnet_output(output_text, key_sentence_map):
    #TODO: Use recursion to go levels deeper. Right now it checks only single level dependency   
    word_dependency = {} 
    sentences_pos_tagged = output_text.split("\n\n\n")
    word_list = []
    # print sentences_pos_tagged

    for key in sorted(key_sentence_map.keys()):
        # print key,":"
        word_dependency[key] = []
        for sentence in key_sentence_map[key]:
            sentence_id = sentence[0]
            word_list = []
            # print "Sentence :", sentence_id
            # print sentences_pos_tagged[sentence_id]
            text_to_parse = sentences_pos_tagged[sentence_id-1]
            for line in text_to_parse.split("\n"):
                if(line and line[0].isdigit()):
                    line = line.replace("_", "")
                    # print "Line:"+ line
                    # word_list[x]           x = 0 id              1 word          2 Pos           3  dep          4 INFO
                    word_list.append(tuple((line.split()[0], line.split()[1], line.split()[2], line.split()[4], line.split()[5])))
            

            for word1 in word_list:
                #      Single word          more than one word
                if( (word1[1] == key or word1[1] in key.split("_")) and (word1[2] in ["NOUN","VERB","ADJ","ADV","PROPN","NUM"])):
                    for word2 in word_list:
                        #If they are not same words AND if word_id of second word = dep_word_id of candidate word
                        if( word2[0]!=word1[0] and word2[1] not in key.split("_") and (word2[0] == word1[3] or word2[3] == word1[0]) and (word2[2] in ["NOUN","VERB","ADJ","ADV","PROPN","NUM"]) ):
                                                            
                                                            #  WORD      POS
                            word_dependency[key].append(tuple((word2[1],word2[2])))

            #Removing duplicates             
            word_dependency[key] = list(set(word_dependency[key]))


    pickle.dump(word_dependency, open("savepoints/word_dependency.p","wb"))
    return word_dependency


####################################################################################################
# Function 
#     Name:   extract_pos
#
# Task:       For each sentence containing a keyword, pass that sentence through syntaxnet 
#             to extract POS tags for each word. For each keyword, if it is a noun or verb,
#             append the adjectives, adverbs and quantifiers which describe them to the
#             candidate graph. If the keyword is an adjective or and adverb, add the noun or
#             verb it describes to the candidate graph.  
#           
# Parameters:-
#             1. transcript file: Described previously.
#             2. candidate_terms: keywords after tf-idf filtering. 


def extract_pos(transcript_file, candidate_terms):
    with open(transcript_file, "r") as f:
        replace_punctuation = string.maketrans(string.punctuation, " "*len(string.punctuation)) 
        transcript_text = f.read().lower()

    transcript_text = transcript_text.translate(replace_punctuation) #Remove punctuations
    transcript_text = transcript_text.decode("ascii","ignore") #Remove non-ordinal values
    transcript_text = re.sub(' +',' ',transcript_text) #Remove extra spaces 
    transcript_text = re.sub("\n+","\n", transcript_text) #Remove extra newlines

    json.dump(transcript_text , open("savepoints/transcript_text","w"), indent=2)

    # candidate_terms = pickle.load(open("savepoints/candidate_terms.p","rb"))
    candidate_terms_expanded = [term.translate(replace_punctuation).split('_') for term in candidate_terms]
    chain = itertools.chain(*candidate_terms_expanded)
    candidate_terms_expanded = list(chain)

    

    transcript_text_lines = transcript_text.split("\n")
    # print transcript_text_lines
    # print "Total number of lines in Transcript:",len(transcript_text_lines)    
    # transcript_text = transcript_text.decode("ascii","ignore").replace("\n\n", " " ) #Remove newline
    json.dump(transcript_text_lines, open("savepoints/transcript_text_lines","wb"), indent=2)


    transcript_text_split = transcript_text.replace("\n","").split(" ") #Create a list with each word as an element (helps to create sentences using windows)
    json.dump(transcript_text_split, open("savepoints/transcript_text_split","wb"), indent=2)

    found = 0
    sentences ={}
    indices = []
    line_no = 0
    sentence_doc_text = ""
    sentence_id = 1  #This is used to indicate which sentence belongs to which keyterm
    key_sentence_map = {}

    for term in candidate_terms: #use sliding window to find keyterms (including multiple words)
        sentences[term] = []
        if '_' in term:
            term_split = term.split('_')
            indices = []
            for i in xrange(len(transcript_text_split)):
                if transcript_text_split[i:i+len(term_split)] == term_split: 
                    indices.append(i)

            # if(len(indices) > 0):
            #     print term,"found."

            for index in indices:
                text = transcript_text_split[max(0, index- SENTENCE_WINDOW ):min(len(transcript_text_split),index+SENTENCE_WINDOW+(len(term_split)) )]
                text = " ".join(text)
                #find line_no of text in transcript
                line_no = 1
                for line in transcript_text_lines:
                    if text in line:
                        sentences[term].append( tuple(( line_no, text )) )
                        break
                    else:
                        line_no += 1

        else:
            indices = [i for i, x in enumerate(transcript_text_split) if x == term]

            # if(len(indices) > 0):
            #     # print term,"found."

            for index in indices:
                text = transcript_text_split[max(0,index-SENTENCE_WINDOW):min(len(transcript_text_split),index+SENTENCE_WINDOW+1)]
                
                text = " ".join(text)
                #find line_no of text in transcript
                line_no = 1
                for line in transcript_text_lines:
                    if (text in line) or (text[int(len(text))/2:] in line) or text[:int(len(text))/2+1] in line:
                        sentences[term].append( tuple((line_no, text)) )
                        break

                    else:
                        line_no += 1

        #Removing duplicates               
        sentences[term] = list(set(sentences[term]))               
                

    # print "\nSearched for " +str(len(candidate_terms))+" candidate terms. "+str(found)+" candidate terms found.\n"
    # print "Sentences:"
    json.dump(sentences, open("savepoints/sentences","w"), sort_keys=True, indent=2)


    for key in sorted(sentences.keys()):
        # print key, ":"
        key_sentence_map[key] = []

        for sentence in sentences[key]:
            line_no = sentence[0]
            text = sentence[1]
            key_sentence_map[key].append( tuple((sentence_id, line_no, text)))
            sentence_id+=1

            # text = " ".join(text)
            sentence_doc_text += text +" .\n\n"
            # print sentence,"\n"
    # json.dump(sentence_doc_text, open("sentence_doc_text_pos","wb"))

    json.dump(key_sentence_map, open("savepoints/key_sentence_map","w"),sort_keys=True, indent=2)
    # print "==================================================SENTENCE DOC=======================================\n", sentence_doc_text, " ===============================================================================================================\n"

    command = "echo \""+str(sentence_doc_text)+"\"  | "+'''bazel-bin/syntaxnet/parser_eval \
--input stdin \
--output stdout-conll \
--model syntaxnet/models/parsey_mcparseface/tagger-params \
--task_context syntaxnet/models/parsey_mcparseface/context.pbtxt \
--hidden_layer_sizes 64 \
--arg_prefix brain_tagger \
--graph_builder structured \
--slim_model \
--batch_size 1024 | bazel-bin/syntaxnet/parser_eval \
--input stdin-conll \
--output stdout-conll \
--hidden_layer_sizes 512,512 \
--arg_prefix brain_parser \
--graph_builder structured \
--task_context syntaxnet/models/parsey_mcparseface/context.pbtxt \
--model_path syntaxnet/models/parsey_mcparseface/parser-params \
--slim_model --batch_size 1024'''+" > "+SYNTAXNET_OUTPUT_FILE

    time1 = time.time()
    # print "\nCalling syntaxnet on \""+"sentence_doc_text"+"\"..."
    os.chdir(SYNTAXNET_LOCATION) 


    p = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    retval = p.wait()
    syntaxnet_output_text = ""
    new_sentence_flag = False
    with open(SYNTAXNET_OUTPUT_FILE.replace("\\",""),"r") as f:
        for l in f.readlines():
            if(l[0].isdigit()):
                if(new_sentence_flag == False):
                    new_sentence_flag = True
                    syntaxnet_output_text += "\n"
                syntaxnet_output_text += l +"\n"
            else:
                new_sentence_flag = False

    # print "=================================================SYNTAXNET OUTPUT==============================\n", syntaxnet_output_text, " ===============================================================================================================\n"
    time2 = time.time()

    print "Syntaxnet script took "+str(round(time2 - time1 , 3))+ " seconds."
    os.chdir(COMCLUSTER_LOCATION) 

    word_dependency = parse_syntaxnet_output(syntaxnet_output_text, key_sentence_map)

    sentiment_output = extract_sentiment(sentence_doc_text.replace("\n\n", " "), sentences, key_sentence_map)


    key_sentence_sentiment = {}  # key : [ tuple([POS], line_no_in_transcript, sentiment)]

    #For all keywords
    for key in word_dependency.keys():
        key_sentence_sentiment[key] = []
        print key, ":"

        #Select each sentence associated with that keyword
        for (sentence_id, line_no, text) in key_sentence_map[key]:
            #Find line number in transcript
            pos_in_sentence = []

            print "\tSentence", sentence_id, ":", text

            #Find all the pos tags in that sentence
            print "\t word_dependency[",key,"]:",word_dependency[key]
            for pos in word_dependency[key]: # You get (word,pos)
                print "\t\tTrying to find pos word", pos
                if pos[0] in text.split():
                    pos_in_sentence.append(pos[0])

            #Find sentiment
            sentiment = parse_sentiment_output(sentiment_output, sentence_id)
            print sentiment

            if(sentiment):
                pos_in_sentence = list(set(pos_in_sentence))
                key_sentence_sentiment[key].append(tuple(( pos_in_sentence, line_no, sentiment)))

    with open("savepoints/key_sentence_sentiment.json","w") as json_file:
        json_file.write("key_sentence_sentiment = ")

    json.dump(key_sentence_sentiment, open("savepoints/key_sentence_sentiment.json","a"), sort_keys=True, indent=4)

    return word_dependency, key_sentence_sentiment






### Step 7: Extract Sentiment tags

For each sentence containing a keyterm we extract the pos tags pertaining to the keyword in that sentence (if any), the line number where it appeared in the transcription text, and the sentiment (very negative, negative, neutral, positive, very positive) annotation for the sentence.

The <a href = "http://nlp.stanford.edu/sentiment/">sentiment annotator</a> from Stanford's <a href="http://stanfordnlp.github.io/CoreNLP/index.html">CoreNLP package</a> was used to attach sentiment labels to sentences. The underlying technology is based on a type of Recursive Neural Network that builds on top of grammatical structures. This model was trained on the Stanford <a href="http://nlp.stanford.edu/sentiment/treebank.html">Sentiment Treebank dataset</a>. The model and dataset are described in this <a href ="http://nlp.stanford.edu/~socherr/EMNLP2013_RNTN.pdf">EMNLP paper</a>.

In [45]:
####################################################################################################
# Function 
#     Name:   parse_sentiment_output
#
# Task:       This method parses the sentiment output to get the sentiment for each sentence_id
#           
# Parameters:-
#             1. sentiment_output: String that contains the XML output.
#             2. sentence_id: ID of sentence in SENTENCE_DOC whose sentiment is to be obtained. 

def parse_sentiment_output(sentiment_output, sentence_id):
    root = ET.fromstring(sentiment_output)

    for child in root.iter("sentence"):
        if("id" in child.attrib.keys()):
            if(child.attrib["id"] == str(sentence_id)):
                return child.attrib["sentiment"]
          

    # print "Sentence ID NOT FOUND! "
    return None


####################################################################################################
# Function 
#     Name:   extract_sentiment
#
# Task:       Use Stanford CoreNLP to extract sentiments from the required sentences.

# Parameters:-
#             1. sentence_doc_text: Period delimited lines.
#             2. sentences: keyword-sentence mapping. 

def extract_sentiment(sentence_doc_text, sentences, key_sentence_map):

    sentiments = {} # {[POS], line_no_in_transcript, sentiment}
    json.dump(sentence_doc_text, open(SENTENCE_DOC,"wb"), indent=2)

    os.chdir(CORENLP_LOCATION)
    command = ''' java -cp "*" -Xmx2g edu.stanford.nlp.pipeline.StanfordCoreNLP \
-annotators tokenize,ssplit,pos,lemma,parse,sentiment \
-file ''' + COMCLUSTER_LOCATION.replace(" ", "\\ ")+"/"+SENTENCE_DOC.replace(" ", "\\ ") + " -outputDirectory " + COMCLUSTER_LOCATION.replace(" ","\\ ") +"/savepoints/"

    p = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    retval = p.wait()

    for line in p.stdout.readlines():
        print line

    os.chdir(COMCLUSTER_LOCATION)

    with open(SENTENCE_DOC+".out","rb") as f: #Need only filename
        sentiment_output = f.read() 

    return sentiment_output
    

## Integrating all components
As the above code mentioned are actually functions, we integrate them all inorder as shown below.

In [46]:
def keyword_extraction(transcript_file, use_wiki_savepoint = True):

    time1 = time.time()
    global wiki_titles
    global category_tree

    if(use_wiki_savepoint):
        wiki_titles, category_tree = wikidata(use_savepoint=use_wiki_savepoint)

    else:
        #TODO: READ WIKI CONFIG HERE
        wiki_titles, category_tree = wikidata(title_location = TITLE_DATA_LOCATION, 
                                              category_tree_location = CATEGORY_TREE_LOCATION,
                                              category_tree_start_line = CATEGORY_TREE_STARTLINE,
                                              use_savepoint=use_wiki_savepoint)

    print "**************************************N-Gram Extraction******************************"
    #Extracting all n-grams that are wikipedia titles
    candidate_terms = extract_ngrams() 

    print "**************************************TF-IDF Filtering*******************************"
    #Selecting top n% of candidate terms based on their tf-idf value 
    candidate_terms = top_n_tfidf(candidate_terms)

    print "**************************************Building Graph Viz******************************"
    #Construct document graph and find communities.
    candidate_graph = build_graph(candidate_terms)





    time2 = time.time()

    if(candidate_graph):
        print "\n\nKeyword Extraction Run Time:"+ str(round(time2-time1, 4)) + " seconds."

    else:

        print "candidate_graph was empty. No communities found"

    return candidate_graph

if __name__ == '__main__':

    #Change use_wiki_savepoints to FALSE only when wiki data is updated.
    print "Keyword Extraction: Running on "+INPUT_TRANSCRIPT
    candidate_graph = keyword_extraction(INPUT_TRANSCRIPT, use_wiki_savepoint=True)
    print "Keyword Extraction module executed."


Keyword Extraction: Running on data/sample_text.txt
Loading Wikipedia Data...
Size of wiki titles in memory: 768.000267029 MB
Sample Title: okari_river

**************************************N-Gram Extraction******************************
284 n-grams extracted:
['gazit', 'be_one', 'settlement', 'religious', 'concerned', 'iraqi', 'resources', 'far', 'long_term', 'is_to', 'continued', 'is_a', 'work_that', 'saudi', 'the_creation', 'prediction', 'the_goal', 'all_the_way', 'even', 'the_sight', 'access', 'public', 'a_talk', 'iran', 'reported', 'active', 'iraq', 'ahead', 'military', 'a_sigh', 'the_need', 'crisis', 'wealth', 'would', 'around_the_world', 'the_man', 'the_family', 'israel', 'out_of_control', 'particular', 'middle', 'the_people', 'is_not', 'a_rival', 'eda_exploratory_data_analysis', 'control', 'fundamentalist', 'media_coverage', 'arabia', 'fundamentalism', 'metlife_india_team', 'huge', 'goal', 'may', 'southern', 'active_support', 'islamic', 'we_can', 'president', 'military_intelli

In [22]:
os.chdir(COMCLUSTER_LOCATION)

## R Code for Visualization 
This code is part of the RMarkdown file generating the meeting report. It generates the graph visualization for keywords extracted from the transcription.<br />
<b> NOTE: </b> <i><b>Do not run this code here</b></i>. Instead run this on the machine where the report is generated (where keyword extraction is running.)

In [ ]:
library(networkD3)
library(htmlwidgets)
library(htmltools)

nodes <- read.csv("/home/iot/M.R. Robot/Ears/savepoints/nodes.csv")
links <- read.csv("/home/iot/M.R. Robot/Ears/savepoints/links.csv")

if(nrow(nodes) !=  0){
  
  browsable(
  
    tagList(
            
              forceNetwork(Links = links, Nodes = nodes,
                       Source = "source", Target = "target",
                       Value = "relation", NodeID = "name",
                       Group = "community", opacity = 1.0, zoom = TRUE, 
                       linkDistance = JS("function(d){return 250 / d.value}"),
                       linkWidth = 0.1, Nodesize = "size",fontSize = 40,fontFamily = "arial",
                       colourScale = "d3.scale.category20()", charge = -2000),
  
              tags$script('document.body.style.backgroundColor = "#000000"')
  
  
           )
         )
}else{
  
  tagList(
  
    tags$br(),
    tags$br(),
    tags$span(style="text-align:center;color:red;font-size:300%;font-weight:bold;","Sorry! No keywords found."),
    tags$script('document.body.style.backgroundColor = "#000000"')
    
  )
            
            
}

## Sample Output From R
Following are the incremental changes to the graph visualization. 

### Iteration I: 19 September 2016 - 23 September 2016 
Below are the visualization features:
* We can see <b>clusters</b> of related keywords.
* A node has a specific <b>colour</b> if it is a candidate keyword or a word describing the candidate keyword (extracted from POS tags).
* A "_pos" suffix is added to all words from sentences (adjectives, adverbs and nouns) referring to the candidate keyword (noun or verb).
* The distances between the nodes represent a product of word2vec similarity and relatedness calculated through wiki data;<br /><br /> <div style = 'text-align:center;'>$Link Distance = \dfrac {Constant} {similarity \times relatedness}$</div><br />  <ul><li><b>similarity</b> : Word similarity calculated using word2vec on google's model. </li><li><b>relatedness</b> : Relatedness based on links, double-links, see-also etc. relations from wiki data.</li><li><b>constant</b>: A constant selected after experimentation so that link distance visualized is aesthetic and informative. 

In [24]:
from IPython.display import IFrame
IFrame('data/23_sep_16_viz.html', width=700, height=350)


### Iteration II: 26 September 2016 - 30 September 2016
Newest features in this iteration is listed below:
* Nodes which carry POS information of keywords are hidden until such keyword node is clicked upon.
* Upon click on a keyword node, all disconnected nodes are hidden.
* POS nodes also carry its specific POS label i.e, if it is a noun, verb, adjective or adverb. 


In [21]:
from IPython.display import IFrame
IFrame('data/27_sep_16_viz.html', width=700, height=350)


### Iteration III: 03 October 2016 - 07 October 2016
* Added a barchart to visualize sentiment evolution of a keyword. 

In [60]:
IFrame('data/Keyword Sentiments _ MR Robot.html', width=1000, height=700
      )

### Iteration IV: 10 October 2016 - 14 October 2016
* Integrated barchart component with meeting report.
* This barchart renders on clicking the keyword node. 
* Each bar represents the sentiment of a sentence spoken at a particular time.
* The sentiments can take the values of 'Very Negative', 'Negative' (red), 'Neutral' (yellow), 'Positive' (green) and 'Very Positive'.
* Hovering over a bar shows all the POS tags associated with the keyword in the sentence spoken at that particular time in the meeting. 
* <b>Note:</b> This is still a work in progress. Many aesthetic features such as axes, borders and render animation  are being worked on.

In [56]:
IFrame('data/14_oct_16_viz.html', width=900, height=700)

### Iteration V: 17th October 2016 - 28th October 2016
* Made changes to the aesthetics and the dynamics of the report
* Added wordcloud to the visualization
* Two wordclouds are shown, one based on Frequency metric and the other based on TF-IDF metric
* The wordclouds help convey an immediate picture of what the topic(s) of discussion might have been
* The midmap graph's barchart that is generated on click was revamped and now better cosmetics
* The entire report can now be seen below


In [9]:
IFrame('data/20_oct_16_viz.html', width=900, height=700)

### Iteration VI: 31st October 2016 To 04th November 2016
* Added speakerwise wordclouds, based on frequency metric, into the meetting attendees section.
* Changed the meeting attendees presentation to grid view.
* Added "Silent Meeting Attendees" section to Meeting attendees for displaying attendees who didn't speak and so did not have a wordcloud visualization.

In [12]:
IFrame('data/04_nov_16_viz.html', width=900, height=700)

## Conclusion
When the keyword extraction algorithm is employed on well-structured sentences (like a blogpost in the examples above), it delivers a set of keywords which presents a generic collection of words and phrases (keyterms), that represent the topics being discussed in the text, to the user. 

The apparent clusters represent related keyterms which are collectively representative of a topic of discussion. The internodal distances expresses degree of relatedness. For example, in the visualization above, we can see that the words Iran, Lebanon and Iraq appear closer to each other within the cluster, giving us the idea that these countries when mentioned in a text have strong contextual relation with one another and the relationship between these countries is probably being discussed. POS tags further depicts the usage of a keyword in a sentence providing further information about the context in which particular keywords were used.  

Further experiments need to be carried out to ascertain the algorithm's effectiveness on the unstructured snippets of text recieved from the audio transcription module.   